In [78]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

In [79]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6623115559224017602
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6940786688
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10616270259764956642
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [80]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [81]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [82]:
print(tf.__version__)

2.8.0


In [83]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [84]:
train_path="output/train"
test_path="output/test"
val_path="output/val"


In [85]:
(height, width, channels) = (80, 48, 3)

x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(height, width))

        x_train.append(img_arr)

x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(height, width))

        x_test.append(img_arr)
        
x_val=[]

for folder in os.listdir(val_path):

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(height, width))

        x_val.append(img_arr)



In [86]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0





In [87]:
#def dubble_image_size(img):
#    size = (48, 80)
#    image = cv2.resize(img, size) # dubble the image size
#    image = image.astype('uint8')
#    equalized_img = cv2.equalizeHist(image)
#  
#    equalized_img = equalized_img.reshape(size[1], size[0], 1)
#    
#    #grayimg = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#    equalized_img = equalized_img.astype('float64')
#    
#     
#    #unsharped_img = unsharp_mask(image, radius=3, amount=1)
#    return equalized_img
#
#
##def dubble_image_size(img):
##   size = (48, 80)
##  return cv2.resize(image,size)    #edw prepei na diabazoume tis diastaseis gia na ginei pio automato

In [88]:
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

val_datagen = ImageDataGenerator(rescale = 1./255)

In [89]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (height, width),
                                                 batch_size = 32,
                                                 class_mode = 'sparse',
                                                 color_mode="grayscale"
                                                )
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (height, width),
                                            batch_size = 32,
                                            class_mode = 'sparse',
                                            color_mode="grayscale")
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (height, width),
                                            batch_size = 32,
                                            class_mode = 'sparse',
                                            color_mode="grayscale")

Found 1524 images belonging to 5 classes.
Found 195 images belonging to 5 classes.
Found 189 images belonging to 5 classes.


In [90]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [91]:
#training_set.class_indices
train_y.shape,test_y.shape,val_y.shape


((1524,), (195,), (189,))

In [92]:
IMAGE_SIZE = [48,80]

In [93]:
ResNet152V2 = ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [94]:
for layer in ResNet152V2.layers:
    layer.trainable = False

In [95]:
x = Flatten()(ResNet152V2.output)


In [96]:
prediction = Dense(5, activation='softmax')(x)

model = Model(inputs=ResNet152V2.input, outputs= prediction)

In [97]:
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 48, 80, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 54, 86, 3)    0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 24, 40, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 26, 42, 64)   0           ['conv1_conv[0][0]']             
                                                                                            

                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 12, 20, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 12, 20, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 12, 20, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_blo

 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 6, 10, 128)  0           ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 6, 10, 512)   66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_out (Add)         (None, 6, 10, 512)   0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_conv[0][0]']    
                                                                                                  
 conv3_blo

                                                                                                  
 conv3_block5_1_relu (Activatio  (None, 6, 10, 128)  0           ['conv3_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block5_2_pad (ZeroPaddin  (None, 8, 12, 128)  0           ['conv3_block5_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block5_2_conv (Conv2D)   (None, 6, 10, 128)   147456      ['conv3_block5_2_pad[0][0]']     
                                                                                                  
 conv3_block5_2_bn (BatchNormal  (None, 6, 10, 128)  512         ['conv3_block5_2_conv[0][0]']    
 ization) 

 ormalization)                                                                                    
                                                                                                  
 conv3_block8_preact_relu (Acti  (None, 6, 10, 512)  0           ['conv3_block8_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv3_block8_1_conv (Conv2D)   (None, 6, 10, 128)   65536       ['conv3_block8_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv3_block8_1_bn (BatchNormal  (None, 6, 10, 128)  512         ['conv3_block8_1_conv[0][0]']    
 ization)                                                                                         
          

...

 conv4_block29_1_bn (BatchNorma  (None, 3, 5, 256)   1024        ['conv4_block29_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block29_1_relu (Activati  (None, 3, 5, 256)   0           ['conv4_block29_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block29_2_pad (ZeroPaddi  (None, 5, 7, 256)   0           ['conv4_block29_1_relu[0][0]']   
 ng2D)                                                                                            
                                                                                                  
 conv4_block29_2_conv (Conv2D)  (None, 3, 5, 256)    589824      ['conv4_block29_2_pad[0][0]']    
          

                                                                                                  
 conv4_block32_preact_bn (Batch  (None, 3, 5, 1024)  4096        ['conv4_block31_out[0][0]']      
 Normalization)                                                                                   
                                                                                                  
 conv4_block32_preact_relu (Act  (None, 3, 5, 1024)  0           ['conv4_block32_preact_bn[0][0]']
 ivation)                                                                                         
                                                                                                  
 conv4_block32_1_conv (Conv2D)  (None, 3, 5, 256)    262144      ['conv4_block32_preact_relu[0][0]
                                                                 ']                               
                                                                                                  
 conv4_blo

 conv4_block34_2_bn (BatchNorma  (None, 3, 5, 256)   1024        ['conv4_block34_2_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block34_2_relu (Activati  (None, 3, 5, 256)   0           ['conv4_block34_2_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block34_3_conv (Conv2D)  (None, 3, 5, 1024)   263168      ['conv4_block34_2_relu[0][0]']   
                                                                                                  
 conv4_block34_out (Add)        (None, 3, 5, 1024)   0           ['conv4_block33_out[0][0]',      
                                                                  'conv4_block34_3_conv[0][0]']   
          

                                                                 ]                                
                                                                                                  
 conv5_block1_1_bn (BatchNormal  (None, 2, 3, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 2, 3, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block1_2_pad (ZeroPaddin  (None, 4, 5, 512)   0           ['conv5_block1_1_relu[0][0]']    
 g2D)                                                                                             
          

                                                                                                  
 conv5_block3_3_conv (Conv2D)   (None, 2, 3, 2048)   1050624     ['conv5_block3_2_relu[0][0]']    
                                                                                                  
 conv5_block3_out (Add)         (None, 2, 3, 2048)   0           ['conv5_block2_out[0][0]',       
                                                                  'conv5_block3_3_conv[0][0]']    
                                                                                                  
 post_bn (BatchNormalization)   (None, 2, 3, 2048)   8192        ['conv5_block3_out[0][0]']       
                                                                                                  
 post_relu (Activation)         (None, 2, 3, 2048)   0           ['post_bn[0][0]']                
                                                                                                  
 flatten_3

In [98]:
model.compile(
  tf.keras.optimizers.Adam(learning_rate=0.06),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

In [99]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=4)

In [100]:
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=50,
  callbacks=[early_stop],
  batch_size=64,shuffle=True)

Epoch 1/50
24/24 [==============================] - 11s 171ms/step - loss: 65.8198 - accuracy: 0.2986 - val_loss: 25.6690 - val_accuracy: 0.3915
Epoch 2/50
24/24 [==============================] - 2s 82ms/step - loss: 22.0666 - accuracy: 0.4744 - val_loss: 23.8772 - val_accuracy: 0.3968
Epoch 3/50
24/24 [==============================] - 2s 82ms/step - loss: 11.7193 - accuracy: 0.5735 - val_loss: 22.4925 - val_accuracy: 0.4762
Epoch 4/50
24/24 [==============================] - 2s 82ms/step - loss: 9.4240 - accuracy: 0.6227 - val_loss: 23.7992 - val_accuracy: 0.3598
Epoch 5/50
24/24 [==============================] - 2s 82ms/step - loss: 7.1539 - accuracy: 0.6785 - val_loss: 18.3500 - val_accuracy: 0.4762
Epoch 6/50
24/24 [==============================] - 2s 82ms/step - loss: 5.8478 - accuracy: 0.7054 - val_loss: 16.9074 - val_accuracy: 0.4444
Epoch 7/50
24/24 [==============================] - 2s 83ms/step - loss: 5.5111 - accuracy: 0.7434 - val_loss: 16.5886 - val_accuracy: 0.4762
E

In [101]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np

In [102]:
y_pred=model.predict(test_x)
y_pred=np.argmax(y_pred,axis=1)

In [103]:
print(classification_report(y_pred,test_y,zero_division=0))


              precision    recall  f1-score   support

           0       0.74      0.48      0.58        52
           1       0.46      0.92      0.61        12
           2       0.27      0.60      0.37        15
           3       0.30      0.76      0.43        17
           4       0.74      0.45      0.56        99

    accuracy                           0.53       195
   macro avg       0.50      0.64      0.51       195
weighted avg       0.65      0.53      0.54       195



In [105]:
confusion_matrix_=confusion_matrix(y_pred,test_y)


In [106]:
confusion_matrix_

array([[25,  3,  9,  3, 12],
       [ 0, 11,  1,  0,  0],
       [ 1,  1,  9,  3,  1],
       [ 0,  0,  1, 13,  3],
       [ 8,  9, 13, 24, 45]], dtype=int64)

In [29]:
import seaborn as sb
from matplotlib.colors import LogNorm, PowerNorm


In [31]:
model.save("fer_ckplus_ResNet152V2_model.h5")
print("Done")

Done
